In [142]:
import pandas as pd
import joblib
import pymorphy2
import yaml
import json
import re
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_extraction.text import TfidfVectorizer

In [132]:
config_path = '../config/params.yml'
config = yaml.load(open(config_path), Loader=yaml.FullLoader)

preproc = config['preprocessing']
training = config['train']
evaluate = config['evaluate']

In [133]:
preproc

{'train_path': '../data/raw/train.csv',
 'unique_values_path': '../data/processed/unique_values.json',
 'train_path_proc': '../data/processed/train.csv',
 'test_size': 0.2,
 'val_size': 0.16,
 'target_column': 'cat',
 'target_column_pred': 'cat_pred',
 'sum_columns': ['title', 'description', 'adomain', 'bundle'],
 'drop_columns': ['text', 'text_count'],
 'rename_columns': {'cat': 'cat_orig'},
 'unnecessary_words': ['google',
  'ru',
  'app',
  'apps',
  'com',
  'android',
  'apple',
  'lv'],
 'vectorizer_params': {'max_df': 0.9, 'max_features': 9000},
 'random_state': 10}

In [134]:
training

{'n_trials': 10,
 'random_state': 10,
 'target_column': 'cat',
 'model_path': '../models/model_svc_optuna.joblib',
 'study_path': '../models/study.joblib',
 'metrics_path': '../report/metrics.json',
 'params_path': '../report/best_params.json'}

In [135]:
evaluate

{'data_origin_path': '../data/check/data_origin.csv',
 'data_num_path': '../data/check/data_num.csv'}

# Import

Загружаем новые, продовские данные

In [136]:
data_eval_origin = pd.read_csv(evaluate['data_origin_path'], index_col=0, keep_default_na=False)
data_eval_origin

,title,description,adomain,bundle
0,Приложения в Google Play Строки от МТС,"Книги, подкасты и аудиокниги",,
1,Мы главные по качеству. Убедитесь в этом сами ...,Мы главные по качеству. Мы гарантируем каждому...,,
2,Dodo Pizza - Apps on Google Play,Pizza delivery takeaway,,
3,"Лента Онлайн доставка продуктов на дом, купить...",pageMetaDescription,,
4,AppStore: ВкусВилл: доставка продуктов,"Читайте отзывы, сравнивайте оценки покупателей...",,
...,...,...,...,...
8285,Disney on the App Store,"Read reviews, compare customer ratings, see sc...",,
8286,Shopee: Mua Sm Online on the AppStore,"Read reviews, compare customer ratings, see sc...",,
8287,-App Store,--iPhoneiPadiPod touch,,
8288,Slickdeals: Deals Discounts - Apps on Google Play,"Get the deals, discounts, coupons - save with ...",,


# Preprocessing

In [137]:
def get_data_text(data: pd.DataFrame, sum_columns: list) -> pd.DataFrame:
    """
    Объединяет текст из нескольких колонок в одну колонку, убирая небуквенные символы
    :param data: исходный датафрейм содержащий текстовые колонки
    :param sum_columns: список колонок для объединения
    :return: датафрейм содержащий одну колонку: 'text', которая содержит объединённый
        текст из колонок sum_columns
    """
    # Создаем пустой DataFrame с колонкой "text"
    data_text = pd.DataFrame(columns=['text'])
    # Добавляем столько же пустых строк, сколько строк в data
    for _ in range(data.shape[0]):
        data_text = data_text.append({'text': ''}, ignore_index=True)
    # Суммируем колонки в одну
    for i in range(len(sum_columns)):
        data_text['text'] += data[sum_columns[i]].astype(str) + ' '
    # Заменяем небуквенные символы на пробелы, а затем множественные пробелы на одинарные
    # и убираем пробелы в начале и в конце строки
    data_text['text'] = data_text['text'].apply(
        lambda x: re.sub(r'\s+', ' ', re.sub(r'[^a-zA-Zа-яА-ЯЁё]', ' ', x)).strip())
    return data_text


def stem(lst: list, unnecessary_words: list) -> None:
    """
    Преобразует значения элементов листа, удаляя ненужные слова и
    приводя оставшиеся слова к нормальной форме
    :param lst: лист для изменения, элементы представляют из себя текстовые предложения
    :unnecessary_words: лист с ненужными словами
    :return: None, т.к. изменяется изначально поданный на функцию лист
    """
    morph = pymorphy2.MorphAnalyzer()

    for i in range(len(lst)):
        for word in unnecessary_words:
            lst[i] = re.sub(rf"\b{word}\b", "", lst[i], flags=re.IGNORECASE)
        lst[i] = " ".join(
            [morph.parse(word)[0].normal_form for word in lst[i].split()]
        )


def tf_idf(list_text: list, vectorizer_params: dict, verbosity: bool=False, csv_path: str=None) -> pd.DataFrame:
    """
    Из листа с текстовыми элементами формирует датафрейм с числовыми признаками
    по алгоритму TF-IDF
    :param list_text: лист с текстовыми признаками
    :param vectorizer_params: параметры для алгоритма TF-IDF
    :param verbosity: нужно ли выводить значения на экран
    :param csv_path: если не None, то путь для сохранения итогового датасета
    :return: датафрейм, названиями признаков которого являются слова из текстовых
        элементов изначального листа, а объектами -- числовые значения, рассчитанные
        по TF-IDF
    """
    vectorizer = TfidfVectorizer(**vectorizer_params)
    df_numbers = vectorizer.fit_transform(list_text).toarray()
    df_words = vectorizer.get_feature_names_out()
    numbers_words = pd.DataFrame(df_numbers, columns=df_words)

    if verbosity:
        print(f"df_numbers[:4] = \n{df_numbers[:4]}\n")
        print(f"df_words[:4] = \n{df_words[:4]}\n")
        print(f"numbers_words.head() = \n{numbers_words.head()}\n")
    
    if csv_path:
        numbers_words.to_csv(csv_path)
    
    return numbers_words


def get_data_num(data: pd.DataFrame) -> pd.DataFrame:
    """
    Создаёт датасет с числовыми признаками методом TF-IDF
    :param data: исходный датасет с текстовыми признаками
    :return: датасет с числовыми признаками, созданными методом TF-IDF
    """
    assert isinstance(data, pd.DataFrame), "Проблема с типом данных"
    # Создаём новый датасет с колонкой text
    data_text = get_data_text(data, preproc["sum_columns"])
    # Формируем питон лист из значений колонки text
    list_text = list(data_text["text"].values)
    # Чтобы в дальнейшем сократить количество признаков и упростить модель,
    # переведём слова в нормальную форму и уберём ненужные
    stem(lst=list_text,
        unnecessary_words=preproc["unnecessary_words"])
    # Применяем TF-IDF для создания датасета с числовыми признаками
    data_num = tf_idf(list_text=list_text,
                      vectorizer_params=preproc["vectorizer_params"],
                      csv_path=evaluate["data_num_path"])
    return data_num


def del_new_add_old_cols(data: pd.DataFrame, old_columns: list) -> None:
    """
    Удаляет из датасета те признаки, которых нет в листе, и
    добавляет с нулевыми значениями те признаки,
    которые есть в листе, но которых нет в датасете
    :param data: исходный датасет
    :param old_columns: список признаков
    :return: None, изменяет исходный датасет
    """
    # Удаляем признаки, которых нет в списке old_columns
    columns_to_remove = [col for col in data.columns if col not in old_columns]
    data.drop(columns_to_remove, axis=1, inplace=True)
    
    # Добавляем признаки из old_columns, которых нет в датасете
    columns_to_add = [col for col in old_columns if col not in data.columns]
    for col in columns_to_add:
        data[col] = 0.0  # Добавляем признак с нулевыми значениями


def check_columns_evaluate(data: pd.DataFrame, train_path_proc: str) -> pd.DataFrame:
    """
    Проверка на наличие признаков из train и упорядочивание признаков согласно train
    :param data: датасет data_num
    :param train_path_proc: путь до датасета train
    :return: датасет с упорядоченными признаками
    """
    # Загружаем только строку с признаками обучающего датасета
    df_0_row = pd.read_csv(train_path_proc, nrows=0, index_col=0)
    # Переводим в список
    column_sequence = df_0_row.columns.tolist()

    if set(column_sequence) != set(data.columns):
        # Если признаки отличаются, то
        # вызываем метод, который
        # удалит новые признаки, которых нет в train, и
        # добавит в eval с нулевыми значениями старые признаки из train, которых нет в eval
        del_new_add_old_cols(data, column_sequence)
    # Упорядочиваем
    return data[column_sequence]


def pipeline_preprocess(data: pd.DataFrame, **kwargs) -> pd.DataFrame:
    """
    Пайплайн по предобработке данных
    :param data: оригинальный датасет
    :return: датасет готовый к предсказанию
    """
    # Создание датасета с числовыми признаками методом TF-IDF
    data_num = get_data_num(data)
    # проверка датасета на совпадение с признаками из train
    data_num = check_columns_evaluate(data_num, kwargs["train_path_proc"])
    data_num.to_csv(kwargs["data_num_checked_cols_path"])

    return data_num

In [143]:
data_eval_num = pipeline_preprocess(data=data_eval_origin, **preproc, **evaluate)

In [146]:
data_eval_num[:4]

,abc,abenteuern,ability,ablians,about,abroad,absolviert,absorbing,accepted,access,...,яндекс,яндексбраузер,яндексмузыка,янина,японский,яркий,ярко,ярмарка,ярославль,яф
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [147]:
data_eval_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8290 entries, 0 to 8289
Columns: 8207 entries, abc to яф
dtypes: float64(8207)
memory usage: 519.1 MB


# Evaluate

In [ ]:
model = joblib.load(training['model_path'])

In [ ]:
target_column_pred = preproc['target_column_pred']
data_eval_num[target_column_pred] = model.predict(data_eval_num.values)

In [16]:
data_eval_num[:4]

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Age_bins,Annual_Premium_bins,Vintage_bins,predict
0,Male,25,1,11,1,less_1_year,0,35786.0,152,53,small,medium,medium,0
1,Male,40,1,28,0,1_2_year,1,33762.0,7,111,medium,medium,medium,1
2,Male,47,1,28,0,1_2_year,1,40050.0,124,199,medium,large,large,1
3,Male,24,1,27,1,less_1_year,1,37356.0,152,187,small,medium,large,0
